In [13]:
import numpy as np
import scipy

In [14]:
def back_substitution(U, y):
    n = U.shape[1]
    x = np.zeros_like(y, dtype=np.double)
    x[-1] = y[-1] / U[-1, -1]
    for i in range(n - 2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]

    return x

def linsolve_qr(A, b):
    num_param = A.shape[1]
    q, r = scipy.linalg.qr(A) # Q*R*p = y
    return back_substitution(r[0:num_param], (q.T @ b)[0:num_param]) # solve for: R*p = Q.T * y


def gauss_newton(f, df, x, tolerance=1e-14, max_step=100):
    step = 0
    error = np.linalg.norm(f(x))
    while error > tolerance and step < max_step:
        # print(step, x, error)
        x += linsolve_qr(df(x), -f(x))
        error = np.linalg.norm(f(x))
        step += 1

    return x

In [15]:
# example find minimum of non-linear function
p1 = [0, 5]
p2 = [4, -1]
p3 = [-2, -3]

def f(p):
    (m, n, r) = p
    return np.array([
        (p1[0]-m)**2 + (p1[1]-n)**2 - r**2,
        (p2[0]-m)**2 + (p2[1]-n)**2 - r**2,
        (p3[0]-m)**2 + (p3[1]-n)**2 - r**2,
    ])

def df(p):
    (m, n, r) = p
    return np.array([
        [2 * (m - p1[0]), 2 * (n - p1[1]), -2 * r],
        [2 * (m - p2[0]), 2 * (n - p2[1]), -2 * r],
        [2 * (m - p3[0]), 2 * (n - p3[1]), -2 * r],
    ])

# search for m, n, R
initial_guess = np.array([1, 1, 1], dtype=np.float64)

(m, n, r) = gauss_newton(f, df, initial_guess)
print((m, n, r))

(0.0909090909090908, 0.7272727272727272, 4.273694281288421)
